In [64]:
!pip3 install pyTigerGraph

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
cd '/content/drive/MyDrive/bioNLP'

/content/drive/MyDrive/bioNLP


In [99]:
import pyTigerGraph as tg
import json
import pandas as pd

hostName = "https://mvie.i.tgcloud.io"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


In [85]:
conn.gsql('''
CREATE VERTEX Sentence(PRIMARY_ID id STRING, text STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Author(PRIMARY_ID id STRING, name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Entity(PRIMARY_ID id STRING, name STRING, other_ids STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Article(PRIMARY_ID id STRING, title STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Relation(PRIMARY_ID type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE DIRECTED EDGE rel(FROM Entity, TO Relation|FROM Relation, TO Entity) WITH REVERSE_EDGE="reverse_rel"
CREATE DIRECTED EDGE WROTE(FROM Author, TO Article) WITH REVERSE_EDGE="reverse_WROTE"
CREATE DIRECTED EDGE HAS_SENTENCE(FROM Article, TO Sentence) WITH REVERSE_EDGE="reverse_HAS_SENTENCE"
CREATE DIRECTED EDGE MENTIONS(FROM Sentence, TO Entity) WITH REVERSE_EDGE="reverse_MENTIONS"

''')

'Successfully created vertex types: [Sentence].\nSuccessfully created vertex types: [Author].\nSuccessfully created vertex types: [Entity].\nSuccessfully created vertex types: [Article].\nSuccessfully created vertex types: [Relation].\nSuccessfully created edge types: [rel].\nSuccessfully created reverse edge types: [reverse_rel].\nSuccessfully created edge types: [WROTE].\nSuccessfully created reverse edge types: [reverse_WROTE].\nSuccessfully created edge types: [HAS_SENTENCE].\nSuccessfully created reverse edge types: [reverse_HAS_SENTENCE].\nSuccessfully created edge types: [MENTIONS].\nSuccessfully created reverse edge types: [reverse_MENTIONS].'

In [86]:
conn.gsql('''
CREATE GLOBAL SCHEMA_CHANGE JOB global_schema_change_303694980 {
      ALTER VERTEX Sentence ADD INDEX text_q0sexm9h0od ON (text);
      ALTER VERTEX Author ADD INDEX name_b9v9xwpkoxe ON (name);
      ALTER VERTEX Entity ADD INDEX name_mbd9zqa5dgf ON (name);
      ALTER VERTEX Article ADD INDEX title_4x42wkkn972 ON (title);
      ALTER VERTEX Relation ADD INDEX type_q0v48u0xfqm ON (type);
    }

CREATE GLOBAL SCHEMA_CHANGE JOB global_schema_change {
      ALTER VERTEX Sentence ADD INDEX text_q0sexm9h0od ON (text);
      ALTER VERTEX Author ADD INDEX name_b9v9xwpkoxe ON (name);
      ALTER VERTEX Entity ADD INDEX name_mbd9zqa5dgf ON (name);
      ALTER VERTEX Article ADD INDEX title_4x42wkkn972 ON (title);
      ALTER VERTEX Relation ADD INDEX type_q0v48u0xfqm ON (type);
    }


''')

'Semantic Check Fails: The job name global_schema_change_303694980 already exists in other objects!\nFailed to create global schema change jobs: [global_schema_change_303694980].'

In [111]:
conn.gsql('''
      

CREATE GRAPH bioKG(Sentence, Author, Entity, Article, Relation, rel, WROTE, HAS_SENTENCE, MENTIONS)
CREATE LOADING JOB load_job_article FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Article VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


CREATE LOADING JOB load_job_article_sentences FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE HAS_SENTENCE VALUES($1, $2) USING SEPARATOR=",", HEADER="true";
    }


CREATE LOADING JOB load_job_authWROTE FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE WROTE VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }

CREATE LOADING JOB load_job_entity FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Entity VALUES($1, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


CREATE LOADING JOB load_job_rel_type FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Relation VALUES($2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


CREATE LOADING JOB load_job_relationship FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE MENTIONS VALUES($4, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE MENTIONS VALUES($4, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE rel VALUES($2 Entity, $3 Relation) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE rel VALUES($3 Relation, $1 Entity) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


CREATE LOADING JOB load_job_sentences FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Sentence VALUES($1, $2) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
    }


CREATE LOADING JOB load_job_author FOR GRAPH bioKG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Author VALUES($1, $2) USING SEPARATOR=",", HEADER="false", EOL="\n", QUOTE="double";
    }

''')

'The graph bioKG is created.\nSuccessfully created loading jobs: [load_job_article].\nSuccessfully created loading jobs: [load_job_article_sentences].\nSuccessfully created loading jobs: [load_job_authWROTE].\nSuccessfully created loading jobs: [load_job_entity].\nSuccessfully created loading jobs: [load_job_rel_type].\nSuccessfully created loading jobs: [load_job_relationship].\nSuccessfully created loading jobs: [load_job_sentences].\nSuccessfully created loading jobs: [load_job_author].'

In [112]:
graphName = "bioKG"
conn.graphname = graphName
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)

In [113]:
df = pd.read_csv('article.csv')
print(df.head(0))
print('######')
df = pd.read_csv('author.csv')
print(df.head(0))
print('######')
df = pd.read_csv('entity.csv')
print(df.head(0))
print('######')
df = pd.read_csv('sentences.csv')
print(df.head(0))
print('######')
df = pd.read_csv('relationship.csv')
print(df.head(0))
print('######')
df = pd.read_csv('authWROTE.csv')
print(df.head(0))
print('######')
df = pd.read_csv('rel_type.csv')
print(df.head(0))
print('######')
df = pd.read_csv('article_sentences.csv')
print(df.head(5))
print('######')

Empty DataFrame
Columns: [Unnamed: 0, id, text]
Index: []
######
Empty DataFrame
Columns: [0, 1, Mohammadreza Ahmadi]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, name, other_ids, type]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, text]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, head, tail, type, source]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, sid, did]
Index: []
######
Empty DataFrame
Columns: [Unnamed: 0, id, type]
Index: []
######
   Unnamed: 0  sid                                                did
0           0    1  95be1713dbb23e86525959e49ea5a196b49f3bbc1ce8c6...
1           1    1  3424add78e6e71cf20906f1fc0bd050ace5cd3d9f71a43...
2           2    1  b0c0e0caafe6a371790c385375c5ab9a5edfd695d152cc...
3           3    1  c3ca44e1a8c1fde494adee19104303a349bec064702c50...
4           4    1  9785b08aa0fecd2e0a28784a7f550f08201c5fafed3e2f...
######


In [114]:
article_file = "article.csv"
res = conn.uploadFile(article_file, fileTag='MyDataSource', jobName='load_job_article')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Article",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [115]:
author_file = "author.csv"
res = conn.uploadFile(author_file, fileTag='MyDataSource', jobName='load_job_author')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 1,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Author",
          "validObject": 1,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [116]:
entity_file = "entity.csv"
res = conn.uploadFile(entity_file, fileTag='MyDataSource', jobName='load_job_entity')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 190,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Entity",
          "validObject": 190,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [117]:
sentences_file = "sentences.csv"
res = conn.uploadFile(sentences_file, fileTag='MyDataSource', jobName='load_job_sentences')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 101,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Sentence",
          "validObject": 101,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [118]:
rel_file = "relationship.csv"
res = conn.uploadFile(rel_file, fileTag='MyDataSource', jobName='load_job_relationship')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 26,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "rel",
          "validObject": 26,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "rel",
          "validObject": 26,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        },
        {
          "typeName": "MENTIONS",
          "validObject": 26,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "inval

In [119]:
aw_file = "authWROTE.csv"
res = conn.uploadFile(aw_file, fileTag='MyDataSource', jobName='load_job_authWROTE')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "WROTE",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [120]:
as_file = "article_sentences.csv"
res = conn.uploadFile(aw_file, fileTag='MyDataSource', jobName='load_job_article_sentences')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [],
      "edge": [
        {
          "typeName": "HAS_SENTENCE",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [121]:
rel_file = "rel_type.csv"
res = conn.uploadFile(aw_file, fileTag='MyDataSource', jobName='load_job_rel_type')
print(json.dumps(res, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 2,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "Relation",
          "validObject": 2,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]
